In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.metrics import confusion_matrix, classification_report

# Load the dataset
#import pandas as pd

df = pd.read_csv('/home/rgukt/TASK1/IMDB Dataset.csv',on_bad_lines='skip',quoting=3)
df = df.sample(n=1000, random_state=42)


In [ ]:

 # Ensure the correct file path is provided

# Encode the sentiment labels
# Replace NAType with 0 in the 'sentiment' column
df['sentiment'] = pd.to_numeric(df['sentiment'].map({'positive': 1, 'negative': 0}), errors='coerce').astype('Int64').fillna(0)

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Clean the 'review' column before tokenization
# Remove or replace empty strings and non-string values with a placeholder.
df['review'] = df['review'].astype(str).replace('', 'empty_string')

# Tokenize the reviews
tokens = tokenizer(df['review'].tolist(), padding=True, truncation=True, return_tensors='pt', max_length=128)

# Train-test split (split tokens and labels)
X_train, X_val, y_train, y_val = train_test_split(tokens['input_ids'], df['sentiment'], test_size=0.2, random_state=42)

# Split the attention masks similarly to input_ids
train_attention_mask, val_attention_mask = train_test_split(tokens['attention_mask'], test_size=0.2, random_state=42)

# Create a custom Dataset class
# ... (rest of the code remains the same)

# Create a custom Dataset class
class IMDBDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        # Convert labels to float tensors
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long) # Change to torch.long
        return item

    def __len__(self):
        return len(self.labels)

# Create training and validation datasets
train_dataset = IMDBDataset({'input_ids': X_train, 'attention_mask': train_attention_mask}, y_train)
val_dataset = IMDBDataset({'input_ids': X_val, 'attention_mask': val_attention_mask}, y_val)

# ... (rest of the code remains the same)

# Load the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',  # Output directory
    num_train_epochs=1,  # Number of training epochs
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=8,  # Batch size for evaluation
    warmup_steps=500,  # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # Strength of weight decay
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=10,  # Log every 10 steps
)

# Initialize the Trainer
trainer = Trainer(
    model=model,  # Model to train
    args=training_args,  # Training arguments
    train_dataset=train_dataset,  # Training dataset
    eval_dataset=val_dataset,  # Validation dataset
    tokenizer=tokenizer,  # Tokenizer
)

# Train the model
trainer.train()

# Save the model and tokenizer
model.save_pretrained('./imdb_model')
tokenizer.save_pretrained('./imdb_model')

# Evaluate the model
predictions = trainer.predict(val_dataset)
pred_labels = predictions.predictions.argmax(-1)

# Print evaluation metrics
print(confusion_matrix(y_val, pred_labels))
print(classification_report(y_val, pred_labels))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipykernel_20850/3304745196.py:63: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/home/rgukt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/tmp/ipykernel_20850/3304745196.py:35: FutureWarning: Series.__getitem__ treating

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
#print(confusion_matrix(y_val, pred_labels))
#print(classification_report(y_val, pred_labels))
print("accuracy score:",accuracy_score(y_val, pred_labels))
print("recall_score:",recall_score(y_val, pred_labels))
print("f1_score:",f1_score(y_val, pred_labels))


accuracy score: 0.995
recall_score: 0.0
f1_score: 0.0


In [ ]:
print(df['sentiment'].value_counts())


sentiment
0    997
1      3
Name: count, dtype: Int64


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Compute weights
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(y_train),
                                     y=y_train)

# Convert to tensor
weights = torch.tensor(class_weights, dtype=torch.float)

# Pass these weights to Trainer
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
   output_dir='./results',  # Output directory
    num_train_epochs=1,  # Number of training epochs
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=8,  # Batch size for evaluation
    warmup_steps=500,  # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # Strength of weight decay
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=10,  # Log every 10 steps
)
# Wrap model with weights
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Redefine trainer with weights
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    #compute_metrics=your_metrics_function,
    # Set loss weights
    data_collator=lambda data: {'input_ids': torch.stack([f['input_ids'] for f in data]),
                                'attention_mask': torch.stack([f['attention_mask'] for f in data]),
                                'labels': torch.tensor([f['labels'] for f in data])}
)



Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-7-6e3fe226155c>:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
#retraining the model
# Train the model again on the balanced dataset
train_dataset = IMDBDataset({'input_ids': X_train, 'attention_mask': train_attention_mask}, y_train)
val_dataset = IMDBDataset({'input_ids': X_val, 'attention_mask': val_attention_mask}, y_val)

trainer.train()  # Re-train the model with the balanced dataset


<ipython-input-4-5228a0491b48>:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long) # Change to torch.long


Step,Training Loss
10,0.415000
20,0.362400
30,0.307400
40,0.210500
50,0.135200
60,0.077100
70,0.037000
80,0.019900
90,0.011200
100,0.039000


TrainOutput(global_step=100, training_loss=0.1614693620055914, metrics={'train_runtime': 542.8895, 'train_samples_per_second': 1.474, 'train_steps_per_second': 0.184, 'total_flos': 21788884272000.0, 'train_loss': 0.1614693620055914, 'epoch': 1.0})

In [ ]:
print(confusion_matrix(y_val, pred_labels))
#print(classification_report(y_val, pred_labels))
print("accuracy score:",accuracy_score(y_val, pred_labels))
print("recall_score:",recall_score(y_val, pred_labels))
print("f1_score:",f1_score(y_val, pred_labels))


[[199   0]
 [  1   0]]
accuracy score: 0.995
recall_score: 0.0
f1_score: 0.0


In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
# Load dataset
df = pd.read_csv('/content/IMDB Dataset.csv', encoding='utf-8', on_bad_lines='skip',quoting=3)

# Drop empty/null reviews
df = df[df['review'].notnull()]
df = df[df['review'].str.strip() != '']

# Encode labels
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})


In [ ]:
# Balance the dataset
positive = df[df['sentiment'] == 1]
negative = df[df['sentiment'] == 0].sample(len(positive), random_state=42)
balanced_df = pd.concat([positive, negative]).sample(frac=1, random_state=42).reset_index(drop=True)


In [ ]:
# Step 4: Tokenize Reviews
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokens = tokenizer(
    balanced_df['review'].tolist(),
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors='pt'
)

labels = torch.tensor(balanced_df['sentiment'].values)



In [ ]:
#📦 Step 5: Create Dataset Objects


from torch.utils.data import Dataset

# Create a custom Dataset class
class IMDBDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        # Convert labels to long tensors for CrossEntropyLoss
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

# Split
train_idx, val_idx = train_test_split(range(len(labels)), test_size=0.2, random_state=42)

train_dataset = IMDBDataset({k: v[train_idx] for k, v in tokens.items()}, labels[train_idx])
val_dataset   = IMDBDataset({k: v[val_idx] for k, v in tokens.items()}, labels[val_idx])


In [ ]:
#Load Model and Define Training Args
# Explicitly set problem_type for binary classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2, problem_type="single_label_classification")

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy='epoch',
    save_strategy='epoch',
)

#train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-24-1f31639985e5>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
<ipython-input-23-34ad37dcc011>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)


Epoch,Training Loss,Validation Loss
1,No log,0.754861
2,0.703300,0.732827


<ipython-input-23-34ad37dcc011>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)


TrainOutput(global_step=12, training_loss=0.6780556440353394, metrics={'train_runtime': 190.9404, 'train_samples_per_second': 0.503, 'train_steps_per_second': 0.063, 'total_flos': 5722665454080.0, 'train_loss': 0.6780556440353394, 'epoch': 2.0})

In [ ]:
predictions = trainer.predict(val_dataset)
preds = torch.argmax(torch.tensor(predictions.predictions), axis=1)

print(confusion_matrix(labels[val_idx], preds))
print(classification_report(labels[val_idx], preds, digits=4))


<ipython-input-23-34ad37dcc011>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)


[[1 6]
 [0 5]]
              precision    recall  f1-score   support

         0.0     1.0000    0.1429    0.2500         7
         1.0     0.4545    1.0000    0.6250         5

    accuracy                         0.5000        12
   macro avg     0.7273    0.5714    0.4375        12
weighted avg     0.7727    0.5000    0.4062        12



In [ ]:
print(confusion_matrix(labels[val_idx], preds))
print("accuracy score:",accuracy_score(labels[val_idx], preds))
print("recall score:",recall_score(labels[val_idx], preds))
print("f1_score:",f1_score(labels[val_idx], preds))

[[1 6]
 [0 5]]
accuracy score: 0.5
recall score: 1.0
f1_score: 0.625


In [ ]:
print(df['sentiment'].value_counts())


sentiment
0.0    34
1.0    30
Name: count, dtype: int64


In [ ]:
!pip install transformers datasets scikit-learn -q



# # Setup and Imports


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

# %%
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np


Data Loading and Initial Preprocessing

In [ ]:
# %% [markdown]
# Data Loading and Initial Preprocessing
# %%
# Load the dataset from CSV
# Use on_bad_lines='skip' to handle potential parsing errors and quoting=3 for QUOTE_NONE
df = pd.read_csv('/content/IMDB Dataset.csv', on_bad_lines='skip', quoting=3)

# --- Fix 1: Explicitly balance the dataset after loading and cleaning ---
# Drop empty/null reviews and empty strings first from the full dataset
df = df[df['review'].notnull()]
df = df[df['review'].astype(str).str.strip() != '']

# Encode the sentiment labels for the cleaned full dataframe
df['sentiment'] = pd.to_numeric(df['sentiment'].map({'positive': 1, 'negative': 0}), errors='coerce').astype('Int64').fillna(0)

# Separate positive and negative reviews
positive_df = df[df['sentiment'] == 1]
negative_df = df[df['sentiment'] == 0]

# Sample from the larger class to match the size of the smaller class, or sample a fixed number from each
# Let's aim for a balanced sample, e.g., 500 positive and 500 negative, totaling 1000
sample_size_per_class = 500 # Define sample size per class
if len(positive_df) >= sample_size_per_class and len(negative_df) >= sample_size_per_class:
    sampled_positive = positive_df.sample(n=sample_size_per_class, random_state=42)
    sampled_negative = negative_df.sample(n=sample_size_per_class, random_state=42)
    sampled_df = pd.concat([sampled_positive, sampled_negative]).sample(frac=1, random_state=42).reset_index(drop=True)
else:
    # Handle case where one class has fewer than sample_size_per_class instances
    print("Warning: One class has fewer instances than the desired sample size per class. Using all available instances from the smaller class.")
    min_class_size = min(len(positive_df), len(negative_df))
    sampled_positive = positive_df.sample(n=min_class_size, random_state=42)
    sampled_negative = negative_df.sample(n=min_class_size, random_state=42)
    sampled_df = pd.concat([sampled_positive, sampled_negative]).sample(frac=1, random_state=42).reset_index(drop=True)


# Print value counts for the sampled and cleaned dataframe
print("Sentiment distribution in sampled and cleaned dataset (Balanced):")
print(sampled_df['sentiment'].value_counts())


Sentiment distribution in sampled and cleaned dataset (Balanced):
sentiment
1    30
0    30
Name: count, dtype: Int64


Data Sampling (5000 samples)



In [ ]:
# %% [markdown]
# Tokenization and Dataset Creation
# %%
# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the reviews from the sampled DataFrame
tokens = tokenizer(
    sampled_df['review'].tolist(), # Use review column from sampled_df
    padding=True,
    truncation=True,
    max_length=128, # Keeping max_length consistent for now
    return_tensors='pt' # Return PyTorch tensors
)
# %%
# Get labels from the sampled_df as a PyTorch tensor
# The sentiment column is now guaranteed to have integer values (0 or 1) after balancing
labels = torch.tensor(sampled_df['sentiment'].values.astype('int64'))

# Print shapes to confirm data is ready
print("Shape of input_ids tensor:", tokens['input_ids'].shape)
print("Shape of attention_mask tensor:", tokens['attention_mask'].shape)
print("Shape of labels tensor:", labels.shape)
# %%

# Create a custom Dataset class
class IMDBDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # Get a single item (input_ids, attention_mask)
        item = {key: val[idx] for key, val in self.encodings.items()}
        # Get the corresponding label and convert to torch.long
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        # The number of items in the dataset is the number of labels
        return len(self.labels)


# %%
# Train-test split (split tokens and labels)
# Split based on indices to ensure corresponding inputs and masks are split together
import numpy as np # Import numpy if not already imported

# Get indices for splitting
train_idx, val_idx = train_test_split(np.arange(len(labels)), test_size=0.2, random_state=42, stratify=labels) # --- Fix 2: Add stratify=labels for stratified split ---

# Use indices to create training and validation datasets
train_dataset = IMDBDataset(
    {key: tokens[key][train_idx] for key in tokens.keys()}, # Select token data using train_idx
    labels[train_idx] # Select labels using train_idx
)
val_dataset = IMDBDataset(
    {key: tokens[key][val_idx] for key in tokens.keys()}, # Select token data using val_idx
    labels[val_idx] # Select labels using val_idx
)

# %%
# Print dataset sizes to verify the split
print("Training dataset size:", len(train_dataset))
print("Validation dataset size:", len(val_dataset))

# Print sentiment distribution in train and validation sets after stratified split
print("\nSentiment distribution in training dataset:")
train_labels_list = [train_dataset[i]['labels'].item() for i in range(len(train_dataset))]
print(pd.Series(train_labels_list).value_counts())

print("\nSentiment distribution in validation dataset:")
val_labels_list = [val_dataset[i]['labels'].item() for i in range(len(val_dataset))]
print(pd.Series(val_labels_list).value_counts())



Shape of input_ids tensor: torch.Size([60, 88])
Shape of attention_mask tensor: torch.Size([60, 88])
Shape of labels tensor: torch.Size([60])
Training dataset size: 48
Validation dataset size: 12

Sentiment distribution in training dataset:
1    24
0    24
Name: count, dtype: int64

Sentiment distribution in validation dataset:
0    6
1    6
Name: count, dtype: int64


<ipython-input-56-19d48138ed5b>:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)


In [ ]:
# Load the BERT model for sequence classification
# Explicitly set problem_type for binary classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2, problem_type="single_label_classification")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# --- Fix 3: Compute class weights and pass to the model or Trainer ---
# We will pass them to the model's forward pass using a custom Trainer or by modifying the model's loss calculation.
# Let's modify the model's forward pass for simplicity within the Trainer structure.
# This often involves passing the weights to the loss function (CrossEntropyLoss) inside the model's forward method.
# However, the standard Hugging Face Trainer doesn't have a direct argument for loss weights in TrainingArguments.
# A common workaround is to use a custom Trainer or adjust the dataset sampling/weighting.
# Given we've balanced the dataset, class weights might be less critical but can still help.
# Let's compute them anyway if needed later or for comparison.
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(train_labels_list),
                                     y=train_labels_list)
weights = torch.tensor(class_weights, dtype=torch.float)
print("\nComputed Class Weights:", weights)

# If running on GPU, move weights to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
weights = weights.to(device)
model.to(device) # Move model to device

# Note: Passing class weights directly to the Trainer is not standard.
# It's typically done by:
# 1. Modifying the model's forward method to use the weights in the loss calculation.
# 2. Using a custom `compute_loss` method in the Trainer.
# 3. Data resampling (which we partly did by balancing).
# For simplicity and using the standard Trainer, we won't directly pass `weights` here.
# The balancing itself should help significantly. If still needed, a custom Trainer is required.



Computed Class Weights: tensor([1., 1.])


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# %%
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',  # Directory to save results and checkpoints
    num_train_epochs=5,  # --- Fix 4: Increased number of training epochs ---
    per_device_train_batch_size=8,  # Batch size per GPU/CPU for training
    per_device_eval_batch_size=8,  # Batch size per GPU/CPU for evaluation
    warmup_steps=500,  # Number of steps for learning rate warmup
    weight_decay=0.01,  # Strength of weight decay
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=10,  # Log training metrics every 10 steps
    eval_strategy='epoch', # Evaluate the model at the end of each epoch
    save_strategy='epoch', # Save a checkpoint at the end of each epoch
    load_best_model_at_end=True, # Load the best model (based on eval_loss) at the end of training
    metric_for_best_model='eval_loss', # Metric to monitor for determining the best model
    greater_is_better=False # For 'eval_loss', smaller is better
    # --- Optional: Adjust learning rate if needed ---
    # learning_rate=5e-5, # Default is 5e-5, can try slightly lower or higher
)

# %%
# Initialize the Trainer
# Note: We are NOT directly passing 'weights' here. The model loss calculation
# within the BertForSequenceClassification handles the loss. Balancing the dataset
# is the primary way we address imbalance with the standard Trainer.
trainer = Trainer(
    model=model,  # The BERT model to train
    args=training_args,  # The training arguments defined above
    train_dataset=train_dataset,  # The dataset for training
    eval_dataset=val_dataset,  # The dataset for validation (evaluation during training)
    tokenizer=tokenizer,  # The tokenizer (used by the Trainer for potential tokenization if needed, though we pre-tokenized)
    # You could add 'compute_metrics' here to get custom metrics during evaluation *during* training
    # compute_metrics=compute_metrics, # Uncomment if you define a compute_metrics function
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-59-8d02de1b8f0c>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Train the model
print("Starting model training...")
trainer.train()
print("Training finished.")



Starting model training...


<ipython-input-56-19d48138ed5b>:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)


Epoch,Training Loss,Validation Loss
1,No log,0.686752
2,0.686800,0.677475
3,0.686800,0.663971
4,0.678500,0.651771
5,0.652400,0.639128


<ipython-input-56-19d48138ed5b>:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
<ipython-input-56-19d48138ed5b>:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
<ipython-input-56-19d48138ed5b>:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
<ipython-input-56-19d48138ed5b>:36: UserWarning: To copy construct from a tensor, it is recommended 

Training finished.


In [ ]:
# Save the trained model and tokenizer
print("Saving model and tokenizer...")
model.save_pretrained('./imdb_model')
tokenizer.save_pretrained('./imdb_model')
print("Model and tokenizer saved to ./imdb_model")

# Evaluate the model on the validation dataset
print("\nEvaluating the model on the validation set...")
predictions = trainer.predict(val_dataset)


Saving model and tokenizer...
Model and tokenizer saved to ./imdb_model

Evaluating the model on the validation set...


<ipython-input-56-19d48138ed5b>:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)


In [ ]:
# Get the predicted class IDs (index of the highest logit)
pred_labels = predictions.predictions.argmax(-1)

# Get the true labels from the validation dataset
# We iterate through the val_dataset to get the original labels
# val_labels_list was already created after stratified split
true_labels = val_labels_list



In [ ]:
# Print evaluation metrics
print("\nEvaluation Metrics:")
print(confusion_matrix(true_labels, pred_labels))
print(classification_report(true_labels, pred_labels, digits=4)) # Use digits for more precision



Evaluation Metrics:
[[4 2]
 [0 6]]
              precision    recall  f1-score   support

           0     1.0000    0.6667    0.8000         6
           1     0.7500    1.0000    0.8571         6

    accuracy                         0.8333        12
   macro avg     0.8750    0.8333    0.8286        12
weighted avg     0.8750    0.8333    0.8286        12



In [ ]:
# Print individual metrics
print("accuracy score:", accuracy_score(true_labels, pred_labels))
print("recall score:", recall_score(true_labels, pred_labels))
print("f1_score:", f1_score(true_labels, pred_labels))


accuracy score: 0.8333333333333334
recall score: 1.0
f1_score: 0.8571428571428571


In [ ]:
# Check the sentiment distribution of the original sampled and cleaned data again (optional)
print("\nSentiment distribution in sampled and cleaned dataset:")
print(sampled_df['sentiment'].value_counts())

# Example of using the trained model for prediction (optional)
# ... (pre


Sentiment distribution in sampled and cleaned dataset:
sentiment
1    30
0    30
Name: count, dtype: Int64
